In [1]:
import czapi.api as api
import pandas as pd
from time import sleep
import ipywidgets

In [2]:
columns = ['team_name','href','hammer_start','score','final_score','hammer_progression','relative_score','guid']
def make_frame(event:api.Event)->pd.DataFrame:
    return pd.concat([pd.DataFrame(data=boxscore,columns=columns) for boxscore in event.get_flat_boxscores()],axis=0,sort=True).reset_index(drop=True)


In [3]:
def explode_frame(df:pd.DataFrame)->pd.DataFrame:
    df['end'] = df.hammer_progression.apply(lambda x: range(1,len(x)+1))
    return df.explode(column=['hammer_progression','relative_score','end']).reset_index(drop=True)

In [4]:
def transform_frame(df:pd.DataFrame)->pd.DataFrame:
    return df.assign(

     final_score = df.final_score.astype(int)
    ,hammer_progression = df.hammer_progression.astype(bool)
    ,hammer_start = df.hammer_start.astype(bool)
    ,relative_score = df.relative_score.astype(int)
    ,ends = df.end.astype(int)
    ,team_id = df.href.str.extract(pat='^.*teamid=(\d*)[#&].*').astype(int)

)


In [5]:
def get_games_played(df:pd.DataFrame)->pd.DataFrame:
    return df.groupby(['team_name','team_id'])['guid'].nunique().sort_values(ascending=False).to_frame().rename({'guid':'games_played'},axis=1)


In [6]:
def get_hammer_to_start_game(df:pd.DataFrame)->pd.DataFrame:

    return df[['team_name','team_id','guid','hammer_start']].drop_duplicates().groupby(['team_name','team_id'])['hammer_start'].mean().sort_values(ascending=False).to_frame().rename({'hammer_start':'pct_start_with_hammer'},axis=1)



In [20]:
def get_points_scored(df:pd.DataFrame)->pd.DataFrame:

    return df[['team_name','team_id','guid','final_score']].drop_duplicates().groupby(['team_name','team_id'])['final_score'].mean().sort_values(ascending=False).to_frame().rename({'final_score':'pts_for'},axis=1)


In [7]:
print('Input CurlingZone event ID here:')
event_id = ipywidgets.widgets.BoundedIntText(min=1,max=10000)
event_id

Input CurlingZone event ID here:


BoundedIntText(value=1, max=10000, min=1)

In [21]:
run = ipywidgets.widgets.Button(description="Run report!")
out = ipywidgets.widgets.Output()
def run_report(change)->api.Event:
    out.clear_output()
    
    
    event = api.Event(cz_event_id = event_id.value,verbose=True,delay=3)
    df = make_frame(event=event)
    df = explode_frame(df=df)
    df = transform_frame(df=df)
    
    # games played
    games_played = get_games_played(df=df)
    out.append_display_data(games_played)
    
    # hammer to start game
    hammer_to_start_game = get_hammer_to_start_game(df=df)
    out.append_display_data(hammer_to_start_game)
    
    # points score
    
    points_scored = get_points_scored(df=df)
    out.append_display_data(points_scored)
    
#     print(df.head())
#     print(df.dtypes)

run.on_click(run_report)

In [23]:
ipywidgets.widgets.VBox([run,out])

Scraping draw 1.
Scraping draw 2.
Scraping draw 3.
Scraping draw 4.
Scraping draw 5.
Scraping draw 6.
Scraping draw 7.
Scraping draw 8.
Scraping draw 9.
Scraping draw 10.
Scraping draw 11.
Scraping draw 12.
Scraping draw 13.
Scraping draw 14.
Scraping draw 15.
Scraping draw 16.


# Old work

In [23]:
event = api.Event(cz_event_id = cz_event_id,verbose=True,delay=3)

Scraping draw 1.
Scraping draw 2.
Scraping draw 3.
Scraping draw 4.
Scraping draw 5.
Scraping draw 6.
Scraping draw 7.
Scraping draw 8.
Scraping draw 9.
Scraping draw 10.
Scraping draw 11.
Scraping draw 12.
Scraping draw 13.
Scraping draw 14.


In [3]:
columns = ['team_name','href','hammer_start','score','final_score','hammer_progression','relative_score','guid']

In [4]:
df = pd.concat([pd.DataFrame(data=boxscore,columns=columns) for boxscore in event.get_flat_boxscores()],axis=0,sort=True).reset_index(drop=True)
df.head()

,final_score,guid,hammer_progression,hammer_start,href,relative_score,score,team_name
0,10,227480867396377110910835548259064596647,"[True, False, True, False, False]",True,event.php?view=Team&eventid=7473&teamid=165362...,"[0, 3, 1, 5, 8]","[3, 0, 4, 3, X]",Glenn Howard
1,2,227480867396377110910835548259064596647,"[False, True, False, True, True]",False,event.php?view=Team&eventid=7473&teamid=165351#1,"[0, -3, -1, -5, -8]","[0, 2, 0, 0, X]",Carter Bryant
2,3,51813473605694254939503559225988435646,"[False, False, False, True, True, True, False,...",False,event.php?view=Team&eventid=7473&teamid=165351...,"[0, 0, 1, 0, -1, -2, -1, -1, 0]","[0, 1, 0, 0, 0, 1, 0, 1, 0]",Sam Steep
3,5,51813473605694254939503559225988435646,"[True, True, True, False, False, False, True, ...",True,event.php?view=Team&eventid=7473&teamid=165351...,"[0, 0, -1, 0, 1, 2, 1, 1, 0]","[0, 0, 1, 1, 1, 0, 0, 0, 2]",John Willsey
4,9,285892458216111860419332685150026763767,"[True, True, False, False, True, False]",True,event.php?view=Team&eventid=7473&teamid=165368...,"[0, 0, 4, 6, 5, 8]","[0, 4, 2, 0, 3, X]",Owen Purcell


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   final_score         124 non-null    object
 1   guid                124 non-null    object
 2   hammer_progression  124 non-null    object
 3   hammer_start        124 non-null    bool  
 4   href                124 non-null    object
 5   relative_score      124 non-null    object
 6   score               124 non-null    object
 7   team_name           124 non-null    object
dtypes: bool(1), object(7)
memory usage: 7.0+ KB


In [6]:
df['ends'] = df.hammer_progression.apply(lambda x: range(1,len(x)+1))
df.head()

,final_score,guid,hammer_progression,hammer_start,href,relative_score,score,team_name,ends
0,10,227480867396377110910835548259064596647,"[True, False, True, False, False]",True,event.php?view=Team&eventid=7473&teamid=165362...,"[0, 3, 1, 5, 8]","[3, 0, 4, 3, X]",Glenn Howard,"(1, 2, 3, 4, 5)"
1,2,227480867396377110910835548259064596647,"[False, True, False, True, True]",False,event.php?view=Team&eventid=7473&teamid=165351#1,"[0, -3, -1, -5, -8]","[0, 2, 0, 0, X]",Carter Bryant,"(1, 2, 3, 4, 5)"
2,3,51813473605694254939503559225988435646,"[False, False, False, True, True, True, False,...",False,event.php?view=Team&eventid=7473&teamid=165351...,"[0, 0, 1, 0, -1, -2, -1, -1, 0]","[0, 1, 0, 0, 0, 1, 0, 1, 0]",Sam Steep,"(1, 2, 3, 4, 5, 6, 7, 8, 9)"
3,5,51813473605694254939503559225988435646,"[True, True, True, False, False, False, True, ...",True,event.php?view=Team&eventid=7473&teamid=165351...,"[0, 0, -1, 0, 1, 2, 1, 1, 0]","[0, 0, 1, 1, 1, 0, 0, 0, 2]",John Willsey,"(1, 2, 3, 4, 5, 6, 7, 8, 9)"
4,9,285892458216111860419332685150026763767,"[True, True, False, False, True, False]",True,event.php?view=Team&eventid=7473&teamid=165368...,"[0, 0, 4, 6, 5, 8]","[0, 4, 2, 0, 3, X]",Owen Purcell,"(1, 2, 3, 4, 5, 6)"


In [7]:
rs = df.explode(column=['hammer_progression','relative_score','ends']).reset_index(drop=True)
rs.head()

,final_score,guid,hammer_progression,hammer_start,href,relative_score,score,team_name,ends
0,10,227480867396377110910835548259064596647,True,True,event.php?view=Team&eventid=7473&teamid=165362...,0,"[3, 0, 4, 3, X]",Glenn Howard,1
1,10,227480867396377110910835548259064596647,False,True,event.php?view=Team&eventid=7473&teamid=165362...,3,"[3, 0, 4, 3, X]",Glenn Howard,2
2,10,227480867396377110910835548259064596647,True,True,event.php?view=Team&eventid=7473&teamid=165362...,1,"[3, 0, 4, 3, X]",Glenn Howard,3
3,10,227480867396377110910835548259064596647,False,True,event.php?view=Team&eventid=7473&teamid=165362...,5,"[3, 0, 4, 3, X]",Glenn Howard,4
4,10,227480867396377110910835548259064596647,False,True,event.php?view=Team&eventid=7473&teamid=165362...,8,"[3, 0, 4, 3, X]",Glenn Howard,5


In [8]:
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   final_score         896 non-null    object
 1   guid                896 non-null    object
 2   hammer_progression  896 non-null    object
 3   hammer_start        896 non-null    bool  
 4   href                896 non-null    object
 5   relative_score      896 non-null    object
 6   score               896 non-null    object
 7   team_name           896 non-null    object
 8   ends                896 non-null    object
dtypes: bool(1), object(8)
memory usage: 57.0+ KB


In [9]:
rs = rs.assign(

     final_score = rs.final_score.astype(int)
    ,hammer_progression = rs.hammer_progression.astype(bool)
    ,hammer_start = rs.hammer_start.astype(bool)
    ,relative_score = rs.relative_score.astype(int)
    ,ends = rs.ends.astype(int)
    ,team_id = rs.href.str.extract(pat='^.*teamid=(\d*)[#&].*').astype(int)

)
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   final_score         896 non-null    int64 
 1   guid                896 non-null    object
 2   hammer_progression  896 non-null    bool  
 3   hammer_start        896 non-null    bool  
 4   href                896 non-null    object
 5   relative_score      896 non-null    int64 
 6   score               896 non-null    object
 7   team_name           896 non-null    object
 8   ends                896 non-null    int64 
 9   team_id             896 non-null    int64 
dtypes: bool(2), int64(4), object(4)
memory usage: 57.9+ KB


In [10]:
rs.loc[rs.guid == 65436016482989547031430941358604059751].sort_values(by=['team_id','ends'])

,final_score,guid,hammer_progression,hammer_start,href,relative_score,score,team_name,ends,team_id


In [11]:
# hammer to start game

rs[['team_name','team_id','guid','hammer_start']].drop_duplicates().groupby(['team_name','team_id'])['hammer_start'].mean().sort_values(ascending=False)



team_name         team_id
Felix Asselin     165350     0.833333
Michael Fournier  165359     0.800000
Owen Purcell      165366     0.750000
Jacob Dobson      165356     0.666667
Tanner Horgan     165361     0.600000
Dayna Deruelle    165355     0.571429
Mike McEwen       165363     0.500000
Sam Steep         165370     0.500000
Fraser Reid       165367     0.500000
Jason Camm        165352     0.500000
John Willsey      165368     0.500000
Rob Ainsley       165349     0.500000
Sam Mooibroek     165365     0.428571
Alex Champ        165353     0.400000
Glenn Howard      165362     0.400000
Pat Ferris        165358     0.400000
Travis Fanset     165357     0.333333
Carter Bryant     165351     0.333333
Landan Rooney     166308     0.250000
Brandon Tippin    165369     0.250000
Greg Smith        165373     0.250000
Cory Heggestad    165360     0.250000
Jordan Chandler   165354     0.200000
Mark Kean         165364     0.000000
Name: hammer_start, dtype: float64

In [12]:
# games played

rs.groupby(['team_name','team_id'])['guid'].nunique().sort_values(ascending=False)


team_name         team_id
Pat Ferris        165358     10
Dayna Deruelle    165355      7
Sam Mooibroek     165365      7
Sam Steep         165370      6
Felix Asselin     165350      6
Fraser Reid       165367      6
Rob Ainsley       165349      6
Jason Camm        165352      6
John Willsey      165368      6
Mike McEwen       165363      6
Alex Champ        165353      5
Michael Fournier  165359      5
Tanner Horgan     165361      5
Jordan Chandler   165354      5
Glenn Howard      165362      5
Mark Kean         165364      4
Landan Rooney     166308      4
Brandon Tippin    165369      4
Owen Purcell      165366      4
Greg Smith        165373      4
Cory Heggestad    165360      4
Jacob Dobson      165356      3
Carter Bryant     165351      3
Travis Fanset     165357      3
Name: guid, dtype: int64

In [13]:
# points scored

rs[['team_name','team_id','guid','final_score']].drop_duplicates().groupby(['team_name','team_id'])['final_score'].mean().sort_values(ascending=False)


team_name         team_id
Owen Purcell      165366     7.000000
Pat Ferris        165358     6.500000
Glenn Howard      165362     6.200000
Tanner Horgan     165361     5.800000
Jason Camm        165352     5.500000
Felix Asselin     165350     5.166667
Rob Ainsley       165349     5.000000
Sam Steep         165370     4.833333
Dayna Deruelle    165355     4.714286
Mike McEwen       165363     4.666667
Fraser Reid       165367     4.666667
Landan Rooney     166308     4.250000
Michael Fournier  165359     4.200000
John Willsey      165368     4.166667
Jordan Chandler   165354     4.000000
Sam Mooibroek     165365     4.000000
Travis Fanset     165357     4.000000
Greg Smith        165373     4.000000
Mark Kean         165364     3.750000
Alex Champ        165353     3.600000
Cory Heggestad    165360     3.250000
Carter Bryant     165351     3.000000
Brandon Tippin    165369     2.750000
Jacob Dobson      165356     1.666667
Name: final_score, dtype: float64

In [14]:
# points against

_ = rs[['team_name','team_id','guid','final_score']].drop_duplicates().merge(rs[['team_name','team_id','guid','final_score']].drop_duplicates(),on='guid',how='left')

_.loc[_.team_id_x != _.team_id_y].groupby(['team_name_x','team_id_x'])['final_score_y'].mean().sort_values(ascending=False)


team_name_x       team_id_x
Carter Bryant     165351       7.666667
Greg Smith        165373       7.250000
Travis Fanset     165357       7.000000
Jacob Dobson      165356       7.000000
Jordan Chandler   165354       6.800000
Alex Champ        165353       6.000000
Landan Rooney     166308       6.000000
Cory Heggestad    165360       5.750000
Sam Steep         165370       5.500000
Brandon Tippin    165369       5.250000
Michael Fournier  165359       5.200000
Mark Kean         165364       4.750000
Jason Camm        165352       4.500000
Sam Mooibroek     165365       4.428571
Fraser Reid       165367       4.333333
Mike McEwen       165363       4.166667
Rob Ainsley       165349       4.166667
Dayna Deruelle    165355       4.000000
Owen Purcell      165366       3.750000
Glenn Howard      165362       3.400000
John Willsey      165368       3.333333
Pat Ferris        165358       3.300000
Tanner Horgan     165361       2.200000
Felix Asselin     165350       2.166667
Name: final_